# HOME ASSIGNMENT #3: SLACK API - TO GSHEET

## 0. Load Modules

In [1]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os
print("done")

done


In [2]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)

In [3]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [4]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

### TODO #1
Hoàn tất đoạn code sau

In [5]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [6]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [7]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


In [8]:
user_df[user_df.name == 'lovelylife.158']

,user_id,name,display_name,real_name,title,phone,is_bot
23,U01UJ9MUADT,lovelylife.158,,Thu Trang,,,False


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

In [10]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'assignment3-gg.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [26]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1pXjtLvF6yLv33htSd-B8UgoLkV99vdAZPJ1Yj3lhoIY' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

Result: https://docs.google.com/spreadsheets/d/1pXjtLvF6yLv33htSd-B8UgoLkV99vdAZPJ1Yj3lhoIY/edit?usp=sharing


## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [13]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [15]:
channel_ls[2] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'id': 'C01CAMNCJJV',
 'name': 'branding-design',
 'is_channel': True,
 'is_group': False,
 'is_im': False,
 'created': 1601867790,
 'is_archived': False,
 'is_general': False,
 'unlinked': 0,
 'name_normalized': 'branding-design',
 'is_shared': False,
 'parent_conversation': None,
 'creator': 'U01AVDY7JET',
 'is_ext_shared': False,
 'is_org_shared': False,
 'shared_team_ids': ['T01B7SGGMLJ'],
 'pending_shared': [],
 'pending_connected_team_ids': [],
 'is_pending_ext_shared': False,
 'is_member': False,
 'is_private': False,
 'is_mpim': False,
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': '', 'creator': '', 'last_set': 0},
 'previous_names': [],
 'num_members': 9}

### TODO#3 
* Tìm id của channel #atom-assignment2

In [39]:
for c in range(len(channel_ls)):
    if channel_ls[c]['name']=='atom-assignment2':
        print(channel_ls[c]['id'])


C021FSDN7LJ


### Slack API: List messages trong 1 channel

In [52]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [53]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [43]:
msg_ls[21]

{'client_msg_id': 'ab6d23e2-f50e-465e-881b-1327d5ce437c',
 'type': 'message',
 'text': 'Em gửi Assignment 1 ạ:\n1. Task 1: <https://docs.google.com/spreadsheets/d/1bjuqHqDIfUZHL7l562HOQ8je7jwzG3cDzp5M-MVpZms/edit?usp=sharing|https://docs.google.com/spreadsheets/d/1bjuqHqDIfUZHL7l562HOQ8je7jwzG3cDzp5M-MVpZms/edit?usp=sharing>\n2. Task 2:\xa0<https://colab.research.google.com/drive/1zWbE-TC1Yw5TBUen4ezrP51sV68ezhfD?usp=sharing|https://colab.research.google.com/drive/1zWbE-TC1Yw5TBUen4ezrP51sV68ezhfD?usp=sharing>\n\n3. Task 3:\xa0<https://github.com/Tenderriver/datacracy-atom--Phan-Thuy-Giang-|https://github.com/Tenderriver/datacracy-atom--Phan-Thuy-Giang->',
 'user': 'U01U6JQB695',
 'ts': '1620063678.002400',
 'team': 'T01B7SGGMLJ',
 'edited': {'user': 'U01U6JQB695', 'ts': '1620065267.000000'},
 'attachments': [{'title': 'Google Colaboratory',
   'title_link': 'https://colab.research.google.com/drive/1zWbE-TC1Yw5TBUen4ezrP51sV68ezhfD?usp=sharing',
   'fallback': 'Google Colaboratory',
  

In [54]:
not_learners_id = ['U01BE2PR6LU']

In [55]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)

2021-05-10 11:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 16:14:31.003600 https://github.com/danhpcv/
2021-05-10 11:35:39.003800 U01UMC08KL2 2 ['U01UTGS5ZNY', 'U01UMC08KL2'] 2021-05-18 15:09:59.000800 https://github.com/auslynnguyen/
2021-05-08 13:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 13:36:43.006000 https://github.com/auslynnguyen/


In [59]:
sub = {'SubTime':[], 'user ID submit':[], 'User ID review':[],'latest reply':[],'link github':[]}
for i in range(len(msg_ls)):
    ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
    user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
    if msg_ls[i]['user'] not in not_learners_id:
        if 'attachments' in msg_ls[i].keys():
            text = msg_ls[i]['text']
            github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
            if len(github_link) > 0: github = github_link[0]
            if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
            if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
            if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
            if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
            print(ts, user, reply_users_count, reply_users, latest_reply, github)

            sub['SubTime'].append(ts)
            sub['user ID submit'].append(user)
            sub['User ID review'].append(reply_users)
            sub['latest reply'].append(latest_reply)
            sub['link github'].append(github)

2021-05-10 11:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 16:14:31.003600 https://github.com/danhpcv/
2021-05-10 11:35:39.003800 U01UMC08KL2 2 ['U01UTGS5ZNY', 'U01UMC08KL2'] 2021-05-18 15:09:59.000800 https://github.com/auslynnguyen/
2021-05-08 13:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 13:36:43.006000 https://github.com/auslynnguyen/
2021-05-04 20:55:46.001800 U01UTGVPE7N 1 ['U01UMBX2X18'] 2021-05-08 13:36:43.006000 https://github.com/hoaintp/
2021-05-04 00:41:18.002400 U01U6JQB695 2 ['U01UMC0N5U2', 'U01U6JQB695'] 2021-05-11 10:10:06.001300 https://github.com/Tenderriver/
2021-05-03 15:51:19.001200 U01V00KHHHP 1 ['U01UMBZHU2W'] 2021-05-15 00:51:26.024600 https://github.com/saturn1101/
2021-04-26 13:53:56.000800 U01UMBZHU2W 2 ['U01UMJ451V1', 'U01UMBZHU2W'] 2021-05-15 00:52:35.025100 https://github.com/saturn1101/
2021-04-24 09:01:27.043200 U01VB63LRNC 1 ['U01UMJ451V1'] 2021-05-11 19:37:14.003100 https://github.com/AnhThuNguyenHuynh/
2021-04-24 08:

In [60]:
user_sub = pd.DataFrame(sub) ## Dùng pandas để convert dictionaries thành bảng
user_sub.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,SubTime,user ID submit,User ID review,latest reply,link github
0,2021-05-10 11:51:46.004700,U01UJ9LG5U5,"[U01USGKQ771, U01V00KHHHP]",2021-05-15 16:14:31.003600,https://github.com/danhpcv/
1,2021-05-10 11:35:39.003800,U01UMC08KL2,"[U01UTGS5ZNY, U01UMC08KL2]",2021-05-18 15:09:59.000800,https://github.com/auslynnguyen/
2,2021-05-08 13:31:59.005200,U01UMJ36QD9,[U01UMBX2X18],2021-05-08 13:36:43.006000,https://github.com/auslynnguyen/
3,2021-05-04 20:55:46.001800,U01UTGVPE7N,[U01UMBX2X18],2021-05-08 13:36:43.006000,https://github.com/hoaintp/
4,2021-05-04 00:41:18.002400,U01U6JQB695,"[U01UMC0N5U2, U01U6JQB695]",2021-05-11 10:10:06.001300,https://github.com/Tenderriver/


### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [65]:
# ws2 = sh.add_worksheet("Assignment#2 Submission",10000,256) #create new worksheet
# set_with_dataframe(ws2,user_sub) # APPEND DATA TO SHEET


Result: https://docs.google.com/spreadsheets/d/1pXjtLvF6yLv33htSd-B8UgoLkV99vdAZPJ1Yj3lhoIY/edit?usp=sharing